In [10]:
import requests
import pandas as pd
from config import constants,api_key
import module

In [5]:
POPULAR_POST_ENDPOINT = "/getTopPopularPosts"

## Making API call to Reddit api

In [ ]:

querystring = {"time":"year"}# required parameter
headers = {
	"X-RapidAPI-Key": api_key.RAPID_API_KEY,
	"X-RapidAPI-Host": constants.RAPID_API_HOST
}
response = requests.get(constants.BASE_URL + POPULAR_POST_ENDPOINT, headers=headers, params=querystring)


In [7]:
data_popular = response.json()

In [ ]:
for post in data_popular['data']['posts']:
    print(post['id'])
    print(post['title'])
    print(post['numComments'])
    print(post['score'])
    print(post['author'])
    print(post['authorId'])
    print(post['goldCount'])
    print(post['domain'])
    print("\n")

## Turning into data frame

In [37]:
popular_posts =[]
for post in data_popular['data']['posts']:
    popular_post= {
        "id":post['id'],
        "title":post['title'],
        "numComments":post['numComments'],
        "score":post['score'],
        "author":post['author'],
        "authorId":post['authorId'],
        "goldCount":post['goldCount'],
        "domain":post['domain']}
    popular_posts.append(popular_post)

popular_posts_df = pd.DataFrame(popular_posts)

In [4]:
popular_posts_df.head()

,id,title,numComments,score,author,authorId,goldCount,domain
0,t3_11ia0od,"Hi, I’m Keanu Reeves, AMA",32997,290221,lionsgate,t2_2yjg9dt0,21,i.redd.it
1,t3_187p59c,YSK MTN DEW GAME FUEL is back! Partnering with...,0,0,MTNDEWGaming,t2_t4otn2sq,0,mountaindew.com
2,t3_144f6xm,📣 Apollo will close down on June 30th. Reddit’...,15438,220416,iamthatis,t2_4mwol,354,self.apolloapp
3,t3_17xjfyu,Left the office & found the neighbors cat in m...,3475,218627,go_fight_kickass,t2_12g6zr,0,i.redd.it
4,t3_10lcvuy,Trust the process guys,4228,218095,ThoumasTurbando,t2_usejv7ts,3,v.redd.it


In [41]:
# Saving csv file
popular_posts_df.to_csv("popular_posts.csv",index=False)

## Get post details

In [44]:
POST_DETAILS_ENDPOINT = "/getPostDetails"

querystring = {"post_id":"t3_144f6xm"}

headers = {
	"X-RapidAPI-Key": api_key.RAPID_API_KEY,
	"X-RapidAPI-Host": constants.RAPID_API_HOST
}

response = requests.get(constants.BASE_URL+POST_DETAILS_ENDPOINT, headers=headers, params=querystring)

post_detail = response.json()

In [114]:
post_text=post_detail['data']['media']['richtextContent']['document']

In [99]:
# Gettin Key error because of nested text
# Trying to understand hierachy
for line in post_text:
    try:
        a =line['c'][0]['t']
    except KeyError:
        for nested_line in line['c'][0]['c']:
            try:
                a= nested_line['t']
            except:
                print(nested_line)
                continue
        continue

{'c': [{'e': 'text', 't': "I don't want Reddit slandering me to internal employees or public people by saying I threatened them when they reality is that they immediately apologized for misunderstanding me."}], 'e': 'par'}


In [ ]:
# Getting the full post text
for line in post_text:
    try:
        print(line['c'][0]['t'])
    except KeyError:
        for nested_line in line['c'][0]['c']:
            try:
                print( nested_line['t'])
            except KeyError:
                print(nested_line['c'][0]['t'])
                continue
        continue

In [118]:
# getting the whole text as a string
post_string=""
for line in post_text:
    try:
        post_string+=line['c'][0]['t']
    except KeyError:
        for nested_line in line['c'][0]['c']:
            try:
                post_string+= nested_line['t']
            except KeyError:
                post_string+=nested_line['c'][0]['t']
                continue
        continue

In [5]:
popular_with_text_df =pd.read_csv('popular_reddit_posts_with_whole_text.csv')

In [7]:
popular_with_text_df.head()

,id,title,numComments,score,author,authorId,goldCount,domain,post_text
0,t3_11ia0od,"Hi, I’m Keanu Reeves, AMA",32997,290237,lionsgate,t2_2yjg9dt0,21,i.redd.it,NaN
1,t3_17xhbye,Score MTN DEW® GAME FUEL® and prepare to take ...,0,119,MTNDEWGaming,t2_t4otn2sq,0,mountaindew.com,NaN
2,t3_144f6xm,📣 Apollo will close down on June 30th. Reddit’...,15438,220413,iamthatis,t2_4mwol,354,self.apolloapp,"Hey all,It's been an amazing run thanks to all..."
3,t3_17xjfyu,Left the office & found the neighbors cat in m...,3475,218630,go_fight_kickass,t2_12g6zr,0,i.redd.it,NaN
4,t3_10lcvuy,Trust the process guys,4228,218095,ThoumasTurbando,t2_usejv7ts,3,v.redd.it,NaN


### Trying to get other posts' whole text

In [130]:
querystring = {"post_id":"t3_13n65xa"}

headers = {
	"X-RapidAPI-Key": api_key.RAPID_API_KEY,
	"X-RapidAPI-Host": constants.RAPID_API_HOST
}

response_2 = requests.get(constants.BASE_URL+POST_DETAILS_END_POINT, headers=headers, params=querystring)

post_detail_2 = response_2.json()

In [131]:
post_detail_2['data']['media']

{'obfuscated': None,
 'richtextContent': {'document': []},
 'rteMode': 'richtext',
 'hlsUrl': 'https://v.redd.it/hh4izuoxm11b1/HLSPlaylist.m3u8?a=1706010255%2CN2VlYmEwM2Q1Njg2Y2M0YTcxOTc2NmMxMjdmZjc1MzYzMTZiZTZlMjVlOTNhMDUxM2ZkM2EzYWQyYWEwNzkwMg%3D%3D&v=1&f=sd',
 'dashUrl': 'https://v.redd.it/hh4izuoxm11b1/DASHPlaylist.mpd?a=1706010255%2CMmQxYTc1NzE5ZDI1MGQ1NTM1NmE3NmVkN2RlM2Y3NmY4OTMyZmRjYzUxYWJiOGIwZGUxN2JmZDBmYjkwYmRiZg%3D%3D&v=1&f=sd',
 'isGif': False,
 'scrubberThumbSource': 'https://v.redd.it/hh4izuoxm11b1/DASH_96.mp4',
 'posterUrl': 'https://external-preview.redd.it/rZ2yn4NXCxL11fG4lgBZOcUkyZPfDdcZZFOFwmvLAdA.png?width=640&crop=smart&format=pjpg&auto=webp&s=9914940de84db96f70ebc6577015a85fcddeb66f',
 'width': 402,
 'height': 720,
 'type': 'video'}

### post text is usually empty because of that it returns nan

In [12]:
SUBREDDIT_POSTS_ENDPOINT = "/getPostsBySubreddit"
querystring = {"subreddit":"datascience","sort":"new"}

headers = {
	"X-RapidAPI-Key": api_key.RAPID_API_KEY,
	"X-RapidAPI-Host": constants.RAPID_API_HOST
}

response = requests.get(constants.BASE_URL+SUBREDDIT_POSTS_ENDPOINT, headers=headers, params=querystring)



In [14]:
data_science_posts = response.json()

In [30]:
ds_posts =[]
for post in data_science_posts['data']['posts']:
    ds_post= {
        "id":post['id'],
        "title":post['title'],
        "numComments":post['numComments'],
        "score":post['score'],
        "author":post['author'],
        "authorId":post['authorId'],
        "goldCount":post['goldCount'],
        "domain":post['domain']}
    ds_posts.append(ds_post)

ds_posts_df = pd.DataFrame(ds_posts)

In [33]:
ds_posts_df.head()

,id,title,numComments,score,author,authorId,goldCount,domain
0,t3_18qbsme,Weekly Entering & Transitioning - Thread 25 De...,5,7,AutoModerator,t2_6l4z3,0,self.datascience
1,t3_18q8m18,If you work with text do you apply any sort of...,9,17,AnxiousEgg6284,t2_89ar1fajx,0,self.datascience
2,t3_18q5xz2,"Should I work over Christmas to fix my code, o...",38,48,ManagementObvious631,t2_9rp766xc1,0,self.datascience
3,t3_18q5vq6,PyTorch LSTM for time series,8,6,medylan,t2_44oxrfns,0,self.datascience
4,t3_16uc8la,What would the world look like without US mili...,1306,1370,call_of_war_player,t2_vj2jjxzt,0,callofwar.com


In [39]:
ds_posts_df.to_csv("data_science_posts.csv", index=False)

In [34]:
def get_post_text(post_text):
    post_string=""
    for line in post_text:
        try:
            post_string+=line['c'][0]['t']
        except KeyError:
            for nested_line in line['c'][0]['c']:
                try:
                    post_string+= nested_line['t']
                except KeyError:
                    post_string+=nested_line['c'][0]['t']
                    continue
            continue
    return post_string

In [37]:
post_1 = data_science_posts['data']['posts'][1]

In [38]:
get_post_text(post_1['media']['richtextContent']['document'])

"I'm coming from an academic background with experience in text analytics/light NLP but that was mainly geared towards research. I worked with text datasets but for applications that are much less fancy than some of the industry things I see happening. (Note, it was not a CS field, it was a social science)One thing I never saw or read about in academic research was spell-checking text. People would scrape from twitter, websites, collect transcripts from videos, etc. Spell-checking the text was never a consideration, or something I ever saw in research papers, but it seems to be a wanted step in industry.Does anyone have packages you recommend or procedures you've found to work for this? I've seen a few packages but they don't look like they could really handle it, and I'm at a loss because short of reading all the text I'm not sure how a programming language would even know something was spelled incorrectly or what to correct it to, but maybe this is a task LLMs can do?"

In [40]:
post_texts = [get_post_text(post['media']['richtextContent']['document']) for post in data_science_posts['data']['posts']]


KeyError: 'c'